<a href="https://colab.research.google.com/github/rlaaudrb1104/2023-2-weblecture-project/blob/main/GraphCodeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
import pandas as pd

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype6.csv')

In [ ]:
df

,CWE ID,CVE ID,CODE,vul,chmod(),assert(),atoi(),atof(),calloc(),errno(),...,array_index_vulnerability,int_usage_count,short_usage_count,long_usage_count,long_long_usage_count,unsigned_int_usage_count,unsigned_short_usage_count,unsigned_long_usage_count,unsigned_long_long_usage_count,line_count
0,CWE-119,CVE-2011-0530,int mainloop(CLIENT *client) {\n\tstruct nbd_r...,1,0,0,0,0,0,0,...,0,2,0,4,2,0,0,2,2,94
1,CWE-119,CVE-2017-7586,"header_put_marker (SF_PRIVATE *psf, int x)\n{...",1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,9
2,CWE-119,CVE-2016-8633,static int fwnet_incoming_packet(struct fwnet_...,1,0,0,0,0,0,0,...,0,7,0,1,0,0,0,1,0,144
3,CWE-119,CVE-2015-8126,"png_set_filter_heuristics(png_structp png_ptr,...",1,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,11
4,CWE-119,CVE-2017-9992,"static int decode_dds1(GetByteContext *gb, uin...",1,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18373,CWE-787,CVE-2017-7866,static int decode_plte_chunk(AVCodecContext *a...,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,23
18374,CWE-787,CVE-2016-9537,"PixarLogDecode(TIFF* tif, uint8* op, tmsize_t ...",0,0,2,0,0,0,0,...,0,2,1,4,0,0,0,4,0,139
18375,CWE-787,CVE-2018-6063,void MojoAudioOutputIPC::CloseStream() {\n DC...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
18376,CWE-787,CVE-2018-6063,void MojoAudioInputIPC::RecordStream() {\n DC...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [ ]:
df.columns

Index(['CWE ID', 'CVE ID', 'CODE', 'vul', 'chmod()', 'assert()', 'atoi()',
       'atof()', 'calloc()', 'errno()', 'exit()', 'fclose()', 'fopen()',
       'free()', 'gets()', 'read()', 'scanf()', 'printf()', 'printk()',
       'write()', 'strcpy()', 'strcmp()', 'memcpy()', 'malloc()', 'kmalloc()',
       'realloc()', 'puts()', 'strcat()', 'sprintf()', 'strtok()', 'signal()',
       'NULL)', 'vulnerable_strcpy', 'vulnerable_scanf', 'vulnerable_printf',
       'buffer_allocation_check', 'use_after_free', 'file_close_vulnerability',
       'array_index_count', 'pointer_count', 'array_index_vulnerability',
       'int_usage_count', 'short_usage_count', 'long_usage_count',
       'long_long_usage_count', 'unsigned_int_usage_count',
       'unsigned_short_usage_count', 'unsigned_long_usage_count',
       'unsigned_long_long_usage_count', 'line_count'],
      dtype='object')

In [ ]:
columns = ['CWE ID', 'CODE', 'vul', 'chmod()', 'assert()', 'atoi()',
       'atof()', 'calloc()', 'errno()', 'exit()', 'fclose()', 'fopen()',
       'free()', 'gets()', 'read()', 'scanf()', 'printf()', 'printk()',
       'write()', 'strcpy()', 'strcmp()', 'memcpy()', 'malloc()', 'kmalloc()',
       'realloc()', 'puts()', 'strcat()', 'sprintf()', 'strtok()', 'signal()',
       'NULL)', 'vulnerable_strcpy', 'vulnerable_scanf', 'vulnerable_printf',
       'buffer_allocation_check', 'use_after_free', 'file_close_vulnerability',
       'array_index_count', 'pointer_count', 'array_index_vulnerability',
       'int_usage_count', 'short_usage_count', 'long_usage_count',
       'long_long_usage_count', 'unsigned_int_usage_count',
       'unsigned_short_usage_count', 'unsigned_long_usage_count',
       'unsigned_long_long_usage_count', 'line_count']
df = df[columns]

In [ ]:
df.columns

Index(['CWE ID', 'CODE', 'vul', 'chmod()', 'assert()', 'atoi()', 'atof()',
       'calloc()', 'errno()', 'exit()', 'fclose()', 'fopen()', 'free()',
       'gets()', 'read()', 'scanf()', 'printf()', 'printk()', 'write()',
       'strcpy()', 'strcmp()', 'memcpy()', 'malloc()', 'kmalloc()',
       'realloc()', 'puts()', 'strcat()', 'sprintf()', 'strtok()', 'signal()',
       'NULL)', 'vulnerable_strcpy', 'vulnerable_scanf', 'vulnerable_printf',
       'buffer_allocation_check', 'use_after_free', 'file_close_vulnerability',
       'array_index_count', 'pointer_count', 'array_index_vulnerability',
       'int_usage_count', 'short_usage_count', 'long_usage_count',
       'long_long_usage_count', 'unsigned_int_usage_count',
       'unsigned_short_usage_count', 'unsigned_long_usage_count',
       'unsigned_long_long_usage_count', 'line_count'],
      dtype='object')

In [ ]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/Graphcodebert-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Graphcodebert-base")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/Graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)

def tokenized(examples):
  return tokenizer(examples['CODE'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='vul',new_column_name='labels')

In [ ]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch",
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.296400,0.222651,0.911317
2,0.217100,0.244290,0.920022
3,0.173000,0.227278,0.930359
4,0.136000,0.195420,0.944505
5,0.104700,0.247794,0.948857
6,0.088300,0.271158,0.943417
7,0.072500,0.231731,0.955386


TrainOutput(global_step=7238, training_loss=0.15542604120174933, metrics={'train_runtime': 3530.1136, 'train_samples_per_second': 32.798, 'train_steps_per_second': 2.05, 'total_flos': 3.04629979895808e+16, 'train_loss': 0.15542604120174933, 'epoch': 7.0})